# Meteo

## Query structure:

api.meteomatics.com/ 2019-09-28T00:00:00Z P10D: P1D /t_2m:C/45.842875,6.863229/csv

2019-09-28T00:00:00Z - start date
P10D - time interval (10 days)
P1D - time step (1 day)

http://api.meteomatics.com/2019-04-01T07:00:00ZP180D:P1D/t_max_2m_24h:C,precip_24h:mm/47.5596,7.5886/csv?model=mix
http://api.meteomatics.com/2019-04-01T07:00:00ZP180D:P1D/t_max_2m_24h:C,precip_24h:mm/47.5596,7.5886/csv?model=mix


    